### PDF转TXT的Python实现

摘自https://blog.csdn.net/shao824714565/article/details/84792089

#### package 

- python2中是pdfminer ，python3中是pdfminer3k
- PDF更像一张图片,像是在一张纸的各个准确的位置上把内容都摆放出来。大部分情况下，没有逻辑结构，比如句子或段落，并且不能自适应页面大小的调整。PDFMiner尝试通过猜测它们的布局来重建它们的结构，但是不保证一定能工作。

#### 解析pdf文件用到的类

- **PDFParser**：从文件中获取数据
- **PDFDocument**：存储文档数据结构到内存中，保存获取的数据，和PDFParser是相互关联的
- PDFPageInterpreter：解析页面内容
- PDFDevice：把解析到的内容翻译为你需要的格式
- PDFResourceManager：存储共享资源，例如字体或图片

![relation](.\Desktop\relation.png)


#### 布局分析

布局分析返回PDF文档中的每个页面LTPage对象。这个对象和页内包含的子对象，形成一个树结构。
- LTPage：表示整个页。可能会含有LTTextBox，LTFigure，LTImage，LTRect，LTCurve和LTLine子对象。
- LTTextBox：表示一组文本块可能包含在一个矩形区域。注意此box是由几何分析中创建，并且不一定表示该文本的一个逻辑边界。它包含LTTextLine对象的列表。使用 get_text（）方法返回的文本内容。
- LTTextLine：包含表示单个文本行LTChar对象的列表。字符对齐要么水平或垂直，取决于文本的写入模式。
- LTChar：单个文本对象。
- LTAnno：在文本中实际的字母表示为Unicode字符串（？）。需要注意的是，虽然一个LTChar对象具有实际边界，LTAnno对象没有，因为这些是“虚拟”的字符，根据两个字符间的关系（例如，一个空格）由布局分析后插入。
- LTImage：表示一个图像对象。嵌入式图像可以是JPEG或其它格式，但是目前PDFMiner没有放置太多精力在图形对象。
- LTLine：代表一条直线。可用于分离文本或附图。
- LTRect：表示矩形。可用于框架的另一图片或数字。
- LTCurve：表示一个通用的Bezier曲线。
![LTPAGE](.\Desktop\LTPAGE.png)

#### 解析pdf文件

![PDF](.\Desktop\PDF-1.png)
![PDF](.\Desktop\PDF-2.png)
![PDF](.\Desktop\PDF-3.png)

In [14]:
import urllib
import importlib,sys
importlib.reload(sys)
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
 
def parse(DataIO, save_path):
    #对应图片1
    #用文件对象创建一个PDF文档分析器
    parser = PDFParser(DataIO)
    #创建一个PDF文档
    doc = PDFDocument()
    #分析器和文档相互连接
    parser.set_document(doc)
    doc.set_parser(parser)
    #提供初始化密码，没有默认为空
    doc.initialize()
    #对应图片2
    #检查文档是否可以转成TXT，如果不可以就忽略
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        #创建PDF资源管理器，来管理共享资源
        rsrcmagr = PDFResourceManager()
        #创建一个PDF设备对象
        laparams = LAParams()
        #将资源管理器和设备对象聚合
        device = PDFPageAggregator(rsrcmagr, laparams=laparams)
        #创建一个PDF解释器对象
        interpreter = PDFPageInterpreter(rsrcmagr, device)
        
        #对应图片3
        #循环遍历列表，每次处理一个page内容
        #doc.get_pages()获取page列表
        for page in doc.get_pages():
            interpreter.process_page(page)
            #接收该页面的LTPage对象（布局分析）
            layout = device.get_result()
            #这里的layout是一个LTPage对象 里面存放着page解析出来的各种对象
            #一般包括LTTextBox，LTFigure，LTImage，LTTextBoxHorizontal等等一些对像
            #想要获取文本就得获取对象的text属性
            for x in layout:
                try:
                    if(isinstance(x, LTTextBoxHorizontal)):
                        with open('%s' % (save_path), 'a') as f:
                            result = x.get_text()
                            print (result)
                            f.write(result + "\n")
                except:
                    print("Failed")


#解析本地PDF文本，保存到本地TXT
with open(r'.\Desktop\pdfminer.pdf','rb') as pdf_html:
    parse(pdf_html, r'.\Desktop\pdfminer.txt')
#解析网络上的PDF，保存文本到本地
# url = "https:"
# pdf_html = urllib.urlopen(url).read()
# DataIO = StringIO(pdf_html)
# parse_pdf(DataIO, r'E:\parse_pdf')

Extracting Text & Images from PDF Files

August 04, 2010

Update: January 29, 2012
I've corrected this code to work with the current version of pdfminer and it's now available as a github repo:
https://github.com/dpapathanasiou/pdfminer-layout-scanner

PDFMiner is a pdf parsing library written in Python by Yusuke Shinyama.

In addition to the pdf2txt.py and dumppdf.py command line tools, there is a way of analyzing the content tree of each page.

Since that's exactly the kind of programmatic parsing I wanted to use PDFMiner for, this is a more complete example, which continues
where the default documentation stops.

This example is still a work-in-progress, with room for improvement.

In the next few sections, I describe how I built up each function, resolving problems I encountered along the way. The impatient can just get
the code here instead.

Basic Framework
Here are the python imports we need for PDFMiner:

from pdfminer.pdfparser import PDFParser, PDFDocument, PDFNoOutlines
from

def parse_lt_objs (lt_objs, page_number, images_folder, text=[]):
    """Iterate through the list of LT* objects and capture the text or image data contained in each"""
    text_content = [] 

    for lt_obj in lt_objs:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            # text
            text_content.append(lt_obj.get_text())
        elif isinstance(lt_obj, LTImage):
            # an image, so save it to the designated folder, and note it's place in the text 
            saved_file = save_image(lt_obj, page_number, images_folder)
            if saved_file:
                # use html style <img /> tag to mark the position of the image within the text
                text_content.append('<img src="'+os.path.join(images_folder, saved_file)+'" />')
            else:
                print >> sys.stderr, "Error saving image on page", page_number, lt_obj.__repr__
        elif isinstance(lt_obj, LTFigure):
            # LTFigure objects are containers for 

def save_image (lt_image, page_number, images_folder):
    """Try to save the image data from this LTImage object, and return the file name, if successful"""
    result = None
    if lt_image.stream:
        file_stream = lt_image.stream.get_rawdata()
        file_ext = determine_image_type(file_stream[0:4])
        if file_ext:
            file_name = ''.join([str(page_number), '_', lt_image.name, file_ext])
            if write_file(images_folder, file_name, lt_image.stream.get_rawdata(), flags='wb'):
                result = file_name
    return result

The save_image() function needs the following two supporting functions defined:

def determine_image_type (stream_first_4_bytes):
    """Find out the image file type based on the magic number comparison of the first 4 (or 2) bytes"""
    file_type = None
    bytes_as_hex = b2a_hex(stream_first_4_bytes)
    if bytes_as_hex.startswith('ffd8'):
        file_type = '.jpeg'
    elif bytes_as_hex == '89504e47':
        file_type = ',png'
 

#### 局限性

- 如果PDF中只含有文本，那么可以完全解析出来，含有的图片无法获取，因为pdfminer只能获取PDF中的文本。
- 如果这个PDF本身就不能提取文本会在"if not doc.is_extractable:raise PDFTextExtractionNotAllowed"抛出异常，处理带有水印PDF会抛出这个异常。
        

### Bert Model

#### 模型介绍

BERT的全称为Bidirectional Encoder Representation from Transformers，是一个预训练的语言表征模型。它强调了不再像以往一样采用传统的单向语言模型或者把两个单向语言模型进行浅层拼接的方法进行预训练，而是采用新的masked language model（MLM），以致能生成深度的双向语言表征。

主要优点：
- 采用MLM对双向的Transformers进行预训练，以生成深层的双向语言表征。
- 预训练后，只需要添加一个额外的输出层进行微调，就可以在各种各样的下游任务中取得很好的表现。在这过程中并不需要对BERT进行任务特定的结构修改。

提出论文：BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding（https://arxiv.org/pdf/1810.04805.pdf ）

#### BERT的结构

##### 总体结构

以往的预训练模型的结构会受到单向语言模型（从左到右或者从右到左）的限制，因而也限制了模型的表征能力，使其只能获取单方向的上下文信息。而BERT利用MLM进行预训练并且采用深层的双向Transformer组件（单向的Transformer一般被称为Transformer decoder，其每一个token（符号）只会注意到到目前往左的token。而双向的Transformer则被称为Transformer encoder，其每一个token会注意到到所有的token来构建整个模型，因此最终生成能融合左右上下文信息的深层双向语言表征。

隐藏了Transformer的详细结构后，我们就可以用一个只有输入和输出的黑盒子来表示它:
![Transformer](.\Desktop\T-1.png)

而Transformer又可以进行堆叠，形成一个更深的神经网络：
![Transformer](.\Desktop\T-2.png)

最终，经过多层Transformer结构的堆叠后，形成BERT的主体结构：
![Transformer](.\Desktop\bert.png)

##### BERT的输入

![input](.\Desktop\input.png)

- token embeddings使用WordPiece嵌入，WordPiece的一种主要的实现方式叫做BPE（Byte-Pair Encoding）双字节编码。BPE的过程可以理解为把一个单词再拆分，比如"loved","loving","loves"这三个单词，其实本身的语义都是“爱”的意思，但是如果我们以单词为单位，那它们就算不一样的词，在英语中不同后缀的词非常的多，就会使得词表变的很大，训练速度变慢，训练的效果也不是太好。BPE算法通过训练，能够把上面的3个单词拆分成"lov","ed","ing","es"几部分，这样可以把词的本身的意思和时态分开，有效的减少了词表的数量。
- positional embeddings用来表示句子中单词的位置信息。
- segement embeddings则是对于句子整体的，不同的句子有不同的项，它用来实现句子级别的任务。


由于BERT是一个预训练模型，其必须要适应各种各样的自然语言任务，因此模型所输入的序列必须有能力包含一句话（文本情感分类，序列标注任务）或者两句话以上（文本摘要，自然语言推断，问答任务）。

为了完成具体的分类任务，除了单词的token之外，作者还在输入的每一个序列开头都插入特定的分类token（[CLS]），该分类token对应的最后一个Transformer层输出被用来起到聚集整个序列表征信息的作用。

为了完成两个句子以上的任务，Bert在序列tokens中把分割token（[SEP]）插入到每个句子后，以分开不同的句子tokens，并为每一个token表征都添加一个可学习的segement embedding来指示其属于句子A还是句子B。


##### BERT的输出

Transformer的特点就是有多少个输入就有多少个对应的输出，C为分类token（[CLS]）对应最后一个Transformer的输出，$T_i$ 则代表其他token对应最后一个Transformer的输出。对于一些token级别的任务（如，序列标注和问答任务），就把$T_i$输入到额外的输出层中进行预测。对于一些句子级别的任务（如自然语言推断和情感分类任务），就把C输入到额外的输出层中，这里也就解释了为什么要在每一个token序列前都要插入特定的分类token。



#### BERT的预训练任务

##### 预训练过程

预训练就是在拿样本数据对模型进行训练之前做的训练。在NLP的下游任务（比如机器翻译、阅读理解等）中，可以使用的样本数据是比较少的，因为这些任务需要的都是经过专门标注的数据，这样就使得拿这些样本数据直接训练出来的模型效果比较一般，因此就需要对模型进行预训练了。

预训练的目的就是，提前训练好这些下游任务中底层的、共性的部分模型，然后再用下游任务各自的样本数据来训练各自的模型，这样就可以极大地加快收敛速度。在这里举一个CV的例子，对于人脸识别和数字识别两种任务，显然它们有很多的区别，然而在用CNN进行训练的时候，其实比较浅的几层网络学习到的特征是很相似的，都是类似于边角线的这类基础特征，因此我们可以认为这几层网络可以共用，如果我们能够预先训练好这部分，那么模型的训练收敛速度将会大大加快。

对于NLP的下游任务，尽管它们的最终目标各不相同，但是它们也有着共同的、也是必须首先要做的东西，那就是要让模型理解文档中的单词和句子，具体说就是将文本中的无法直接计算的单词转变为可以计算的向量或者矩阵等形式，并且这些数字化的向量要能够比较好地反映出对应单词在句子中的含义，这就NLP中预训练的目的。在NLP中采用的是语言模型来做预训练，从最初的word embedding到ELMO、再到GPT，以及现在的BERT，其实做的都是上面说的这件事，只不过效果变得越来越好。

NLP领域可以利用大规模文本数据的自监督性质来构建预训练任务。因此BERT构建了两个预训练任务，分别是**Masked Language Model(MLM)** 和**Next Sentence Prediction(NSP)**。

##### Masked Language Model(MLM)

MLM是BERT能够不受单向语言模型所限制的原因。简单来说就是以15%的概率用mask token（[MASK]）随机地对每一个训练序列中的token进行替换，然后预测出[MASK]位置原有的单词。然而，由于[MASK]并不会出现在下游任务的微调（fine-tuning）阶段，因此预训练阶段和微调阶段之间产生了不匹配（这里很好解释，就是预训练的目标会令产生的语言表征对[MASK]敏感，但是却对其他token不敏感）。因此BERT采用了以下策略来解决这个问题：

首先在每一个训练序列中以15%的概率随机地选中某个token位置用于预测，假如是第i个token被选中，则会被替换成以下三个token之一：

1）80%的时候是[MASK]。如：my dog is hairy——>my dog is [MASK]

2）10%的时候是随机的其他token。如：my dog is hairy——>my dog is apple

3）10%的时候是原来的token（保持不变，个人认为是作为2中所对应的负类）。如：my dog is hairy——>my dog is hairy

再用该位置对应的 $T_i$ 去预测出原来的token（输入到全连接，然后用softmax输出每个token的概率，最后用交叉熵计算loss）。

该策略令到BERT不再只对[MASK]敏感，而是对所有的token都敏感，以致能抽取出任何token的表征信息。这里给出论文中关于该策略的实验数据：

![MLM](.\Desktop\MLM.png)

##### Next Sentence Prediction(NSP)

一些如问答、自然语言推断等任务需要理解两个句子之间的关系，而MLM任务倾向于抽取token层次的表征，因此不能直接获取句子层次的表征。为了使模型能够有能力理解句子间的关系，BERT使用了NSP任务来预训练，简单来说就是预测两个句子是否连在一起。具体的做法是：对于每一个训练样例，我们在语料库中挑选出句子A和句子B来组成，50%的时候句子B就是句子A的下一句（标注为IsNext），剩下50%的时候句子B是语料库中的随机句子（标注为NotNext）。接下来把训练样例输入到BERT模型中，用[CLS]对应的C信息去进行二分类的预测。

例如：

Input1=[CLS] the man went to [MASK] store [SEP] he bought a gallon [MASK] milk [SEP]

Label1=IsNext

Input2=[CLS] the man [MASK] to the store [SEP] penguin [MASK] are flight ##less birds [SEP]

Label2=NotNext

#### BERT的微调（fine-tuning）

通过上面的方法，我们就实现了BERT模型的预训练，对于特定下游任务的模型，可以通将BERT与一个额外的输出层结合而形成，这样需要重新学习的参数量就会比较小，如下图所示，只需要按BERT模型要求的格式输入训练数据，就可以完成不同的下游任务。

![fine-tuning](.\Desktop\fine-tuning.png)

#### 代码实现

##### packages

- keras_bert：是 CyberZHG 封装好了Keras版的Bert，可以直接调用官方发布的预训练权重，需要keras版本大于等于2.4.3。
- bert4keras：keras4bert 是基于 keras-bert 重新编写的一个 keras 版的 bert，可以适配 albert，只需要在 build_bert_model 函数里加上model='albert'，使用体验和 keras_bert 差不多，需要keras版本小于等于2.3.1。
- 以下代码部分使用keras_bert完成。
- 需要提前下载TensorFlow。

##### Tokenizer

在 keras-bert 里面，使用 Tokenizer 会将文本拆分成字并生成相应的id。

我们需要提供一个字典，字典存放着 token 和 id 的映射，以及 BERT 里特别的 token。

下面是一个例子：

In [16]:
from keras_bert import Tokenizer

# 创建字典
token_dict = {
    '[CLS]': 0,
    '[SEP]': 1,
    'un': 2,
    '##beauti': 3,
    '##ful': 4,
    '[UNK]': 5,
}

tokenizer = Tokenizer(token_dict)

In [17]:
# 拆分单词
print(tokenizer.tokenize('unbeautiful'))

['[CLS]', 'un', '##beauti', '##ful', '[SEP]']


In [18]:
# indices 表示字对应的索引
# segements 表示索引对应位置上的字属于第几句话
# 此处只有一个单词，也就是只有一句话，所以segements均为0
indices, segements = tokenizer.encode('unbeautiful')
print(indices)
print(segements)

[0, 2, 3, 4, 1]
[0, 0, 0, 0, 0]


我们用同样的字典，拆分不存在字典中的单词，结果如下，可以看到英语中会直接把不存在字典中的部分直接按字母拆分。



In [19]:
print(tokenizer.tokenize('unknown'))
indices, segements = tokenizer.encode('unknown')
print(indices)
print(segements)

['[CLS]', 'un', '##k', '##n', '##o', '##w', '##n', '[SEP]']
[0, 2, 5, 5, 5, 5, 5, 1]
[0, 0, 0, 0, 0, 0, 0, 0]


如果输入两句话的例子，encode 函数中 我们可以带上参数 max_len，只看文本拆分出来的 max_len 个字

如果拆分完的字不超过max_len，则用 0 填充

In [20]:
print(tokenizer.tokenize(first = 'unbeautiful', second = 'unknown'))
indices, segements = tokenizer.encode(first = 'unbeautiful', second = 'unknown', max_len = 15)
print(indices)
print(segements)

['[CLS]', 'un', '##beauti', '##ful', '[SEP]', 'un', '##k', '##n', '##o', '##w', '##n', '[SEP]']
[0, 2, 3, 4, 1, 2, 5, 5, 5, 5, 5, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]


##### 使用预训练模型

我们可以使用 load_trained_model_from_checkpoint() 函数使用本地已经下载好的预训练模型，可以从 BERT 的 github 上获取下载地址：

- 谷歌BERT地址：https://github.com/google-research/bert
- 中文预训练BERT-wwm：https://github.com/ymcui/Chinese-BERT-wwm

In [21]:
import os
 
# 设置预训练模型的路径
pretrained_path = '.\Desktop\chinese_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')
 
# 构建字典
# keras_bert 中的 load_vocabulary() 函数 传入 vocab_path 即可
from keras_bert import load_vocabulary
token_dict = load_vocabulary(vocab_path)
 
# import codecs
# token_dict = {}
# with codecs.open(vocab_path, 'r', 'utf8') as reader:
#     for line in reader:
#         token = line.strip()
#         token_dict[token] = len(token_dict)
 
# Tokenization
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)

# 加载预训练模型
from keras_bert import load_trained_model_from_checkpoint
model = load_trained_model_from_checkpoint(config_path, checkpoint_path)

In [22]:
text = '语言模型'
tokens = tokenizer.tokenize(text)

indices, segments = tokenizer.encode(first=text, max_len=512)
print(indices[:10])
print(segments[:10])
 
# 提取特征
import numpy as np
predicts = model.predict([np.array([indices]), np.array([segments])])[0]
for i, token in enumerate(tokens):
    print(token, predicts[i].tolist()[:5])

[101, 6427, 6241, 3563, 1798, 102, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[CLS] [-0.6339440941810608, 0.2029208391904831, 0.08104995638132095, -0.03268882632255554, 0.5675359964370728]
语 [-0.7589294910430908, 0.09625153243541718, 1.0723156929016113, 0.006224863231182098, 0.6886612176895142]
言 [0.549795389175415, -0.7931230664253235, 0.4425913393497467, -0.7123056054115295, 1.2054001092910767]
模 [-0.2921682596206665, 0.6063656210899353, 0.4984249174594879, -0.4249313473701477, 0.42672020196914673]
型 [-0.7458059191703796, 0.49491360783576965, 0.7189143896102905, -0.8728531002998352, 0.8354967832565308]
[SEP] [-0.875252902507782, -0.21610864996910095, 1.3399081230163574, -0.10673174262046814, 0.3961647152900696]


In [23]:
text1 = '语言模型'
text2 = '你好'
tokens1 = tokenizer.tokenize(text1)
print(tokens1)
tokens2 = tokenizer.tokenize(text2)
print(tokens2)
 
indices_new, segments_new = tokenizer.encode(first=text1, second=text2 ,max_len=512)
print(indices_new[:10])
print(segments_new[:10])
 
# 提取特征
import numpy as np
predicts_new = model.predict([np.array([indices_new]), np.array([segments_new])])[0]
for i, token in enumerate(tokens1):
    print(token, predicts_new[i].tolist()[:5])
for i, token in enumerate(tokens2):
    print(token, predicts_new[i].tolist()[:5])

['[CLS]', '语', '言', '模', '型', '[SEP]']
['[CLS]', '你', '好', '[SEP]']
[101, 6427, 6241, 3563, 1798, 102, 872, 1962, 102, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
[CLS] [-0.3404940366744995, 0.5169008374214172, 0.8958086371421814, -0.5850769281387329, 0.16207797825336456]
语 [-0.6919724345207214, 0.37331491708755493, 1.3196660280227661, -0.08652088046073914, 0.5522887706756592]
言 [0.6706030368804932, -0.5946149826049805, 0.47515609860420227, -0.7590193748474121, 0.9860217571258545]
模 [-0.42274874448776245, 0.7286521792411804, 0.5555981993675232, -0.43479907512664795, 0.3921997547149658]
型 [-0.5974085330963135, 0.5976639986038208, 0.7734528183937073, -1.0439555644989014, 0.8142779469490051]
[SEP] [-1.1663368940353394, 0.5416533350944519, 1.3963817358016968, 0.014762148261070251, -0.2048124074935913]
[CLS] [-0.3404940366744995, 0.5169008374214172, 0.8958086371421814, -0.5850769281387329, 0.16207797825336456]
你 [-0.6919724345207214, 0.37331491708755493, 1.3196660280227661, -0.08652088046073914, 0.55

In [26]:
#加载语言模型
model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)
 
token_dict_rev = {v: k for k, v in token_dict.items()}
 
token_ids, segment_ids = tokenizer.encode(u'数学是利用符号语言研究数量、结构、变化以及空间等概念的一门学科', max_len=512)
# mask掉“数学”
token_ids[1] = token_ids[2] = tokenizer._token_dict['[MASK]']
masks = np.array([[0, 1, 1] + [0] * (512 - 3)])
 
# 模型预测被mask掉的部分
predicts = model.predict([np.array([token_ids]), np.array([segment_ids]), masks])[0]
pred_indice = predicts[0][1:3].argmax(axis=1).tolist()
print('Fill with: ', list(map(lambda x: token_dict_rev[x], pred_indice)))

Fill with:  ['数', '学']


#### FinBERT

提出论文：FinBERT: Financial Sentiment Analysis with Pre-trained Language Models（https://openreview.net/pdf?id=HylznxrYDr ）首次将BERT应用于金融领域，提出了FinBERT。

背景：尽管当前很多情感分析方法在商品评论或者电影评论能获得很好的效果，但是这些方法在一些特定领域诸如金融，这些方法的效果还是远远落后。主要是有两方面原因，一是这些领域有许多特定的专用词，二是缺乏大规模高质量的标注数据。

作者在一个更庞大的通用的金融语料集上预训练Bert,然后在特定的分类金融语料集上对上一步预训练好的Bert进行微调。具体过程如下图所示，具体的分类任务可以直接加到BERT模型的下游。

![finbert](.\Desktop\finbert.png)

预训练Bert的数据集介绍：作者构建的一个新的数据集TRC2-financial，它是在Reuters的RC2中根据一些金融关键词过滤得到的一个子集，包括46143个文档，超过29000000个单词和接近400000条句子。文章主要的情感分析数据集是Financial PhraseBank，是从LexisNexis数据库里的金融新闻中随机选中得到的4845条句子，对应的的标签由16个金融和商业背景的人打出，具体分布情况如下表所示，可以看到不同人对不同新闻判断是由一定差异的，只有不到50%的新闻所有人的观点是一致的。另一个数据集FiQA包括1174个金融新闻标题和对应得情感得分，对应得分从-1到1，其中1表示最积极。

![DISTRIBUTION](.\Desktop\DISTRIBUTION.png)
